In [1]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import gc
import numpy as np

import tensorflow as tf

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import merlin.models.tf as mm

2023-03-13 20:54:36.957592: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")
2023-03-13 20:54:39.345898: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 20:54:39.346296: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 20:54:39.346453: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgr

In [2]:
train = Dataset("ecom_dataset/0001/train.parquet")
valid = Dataset("ecom_dataset/0002/test.parquet")

In [3]:
target = 'sess_pid_seq'

In [4]:
# a couple of starter hyperparams

d_model = 192
n_layer = 3
n_head = 16
batch_size = 128
learning_rate = 0.0006667377132554976
n_epoch = 5
item_embedding_dim = 448 
item_id_embeddings_init_std = 3

In [5]:
def get_model():
    mlp_block = mm.MLPBlock(
                    [d_model],
                    activation='relu',
                    no_activation_last_layer=True,
                )

    from merlin.schema.io.tensorflow_metadata import TensorflowMetadata

    schema = TensorflowMetadata.from_proto_text_file(
        '../',
        file_name='rees46_schema_modified.pbtxt'
    ).to_merlin_schema()

    train.schema = schema
    
    schema_model = schema.select_by_tag(Tags.ITEM_ID)
    input_block = mm.InputBlockV2(
        schema_model,
        categorical=mm.Embeddings(
                schema_model.select_by_tag(Tags.CATEGORICAL),
                dim=item_embedding_dim,
                sequence_combiner=None,
            )
        )

    train.schema = train.schema.select_by_name('sess_pid_seq')

    xlnet_block = mm.XLNetBlock(d_model=d_model, n_head=n_head, n_layer=n_layer)

    dense_block = mm.SequentialBlock(
        input_block,
        mlp_block,
        xlnet_block
    )

    mlp_block2 = mm.MLPBlock(
                    [item_embedding_dim],
                    activation='relu',
                    no_activation_last_layer=True,
                )

    prediction_task = mm.CategoricalOutput(
        to_call=input_block["categorical"][target],
    )

    model_transformer = mm.Model(dense_block, mlp_block2, prediction_task)

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
    )

    model_transformer.compile(run_eagerly=False, optimizer=optimizer, loss="categorical_crossentropy",
                  metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[20])
                 )
    return model_transformer, xlnet_block

# Run 1

In [11]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_5/sequential_block_29/xl_net_block_5/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_5/sequential_block_29/xl_net_block_5/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_5/sequential_block_29/xl_net_block_5/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 150s 214ms/step - loss: 8.4001 - recall_at_20: 0.0827 - mrr_at_20: 0.0323 - ndcg_at_20: 0.0433 - map_at_20: 0.0323 - precision_at_20: 0.0041 - regularization_loss: 0.0000e+00 - loss_batch: 8.3857
Epoch 2/5
677/677 [==============================] - 159s 234ms/step - loss: 3.2173 - recall_at_20: 0.6874 - mrr_at_20: 0.5632 - ndcg_at_20: 0.5917 - map_at_20: 0.5632 - precision_at_20: 0.0344 - regularization_loss: 0.0000e+00 - loss_batch: 3.2233
Epoch 3/5
677/677 [==============================] - 159s 235ms/step - loss: 2.0390 - recall_at_20: 0.8298 - mrr_at_20: 0.7342 - ndcg_at_20: 0.7561 - map_at_20: 0.7342 - precision_at_20: 0.0415 - regularization_loss: 0.0000e+00 - loss_batch: 2.0462
Epoch 4/5
677/677 [==============================] - 160s 235ms/step - loss: 1.5995 - recall_at_20: 0.8662 - mrr_at_20: 0.7825 - ndcg_at_20: 0.8016 - map_at_20: 0.7825 - precision_at_20: 0.0433 - regularization_loss: 0.0000e+00 - loss_batch: 1.6068
Epoch 5/5
84/8

{'loss': 8.928336143493652,
 'recall_at_20': 0.32508933544158936,
 'mrr_at_20': 0.11867032200098038,
 'ndcg_at_20': 0.16441309452056885,
 'map_at_20': 0.11867032200098038,
 'precision_at_20': 0.016254469752311707,
 'regularization_loss': 0.0,
 'loss_batch': 10.598859786987305}

In [12]:
model_transformer.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_block_29 (Sequen  multiple                 176252608 
 tialBlock)                                                      
                                                                 
 sequential_block_30 (Sequen  multiple                 86464     
 tialBlock)                                                      
                                                                 
 sess_pid_seq/categorical_ou  multiple                 175110449 
 tput (CategoricalOutput)                                        
                                                                 
 model_context_5 (ModelConte  multiple                 0         
 xt)                                                             
                                                                 
 prepare_features_11 (Prepar  multiple                 0   

# Run 2

In [7]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:149: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_1/sequential_block_9/xl_net_block_1/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 105s 146ms/step - loss: 7.2092 - recall_at_20: 0.1524 - mrr_at_20: 0.0873 - ndcg_at_20: 0.1018 - map_at_20: 0.0873 - precision_at_20: 0.0076 - regularization_loss: 0.0000e+00 - loss_batch: 7.2024
Epoch 2/5
677/677 [==============================] - 100s 147ms/step - loss: 2.8315 - recall_at_20: 0.7410 - mrr_at_20: 0.6393 - ndcg_at_20: 0.6625 - map_at_20: 0.6393 - precision_at_20: 0.0370 - regularization_loss: 0.0000e+00 - loss_batch: 2.8376
Epoch 3/5
677/677 [==============================] - 100s 147ms/step - loss: 1.9715 - recall_at_20: 0.8361 - mrr_at_20: 0.7449 - ndcg_at_20: 0.7658 - map_at_20: 0.7449 - precision_at_20: 0.0418 - regularization_loss: 0.0000e+00 - loss_batch: 1.9781
Epoch 4/5
677/677 [==============================] - 100s 148ms/step - loss: 1.5735 - recall_at_20: 0.8706 - mrr_at_20: 0.7848 - ndcg_at_20: 0.8044 - map_at_20: 0.7848 - precision_at_20: 0.0435 - regularization_loss: 0.0000e+00 - loss_batch: 1.5798
Epoch 5/5
84/8

{'loss': 8.892891883850098,
 'recall_at_20': 0.3253714442253113,
 'mrr_at_20': 0.11890144646167755,
 'ndcg_at_20': 0.16443441808223724,
 'map_at_20': 0.11890144646167755,
 'precision_at_20': 0.016268571838736534,
 'regularization_loss': 0.0,
 'loss_batch': 11.120135307312012}

# Run 3

In [8]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_2/sequential_block_14/xl_net_block_2/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 105s 146ms/step - loss: 7.7130 - recall_at_20: 0.1178 - mrr_at_20: 0.0575 - ndcg_at_20: 0.0708 - map_at_20: 0.0575 - precision_at_20: 0.0059 - regularization_loss: 0.0000e+00 - loss_batch: 7.7064
Epoch 2/5
677/677 [==============================] - 100s 147ms/step - loss: 2.9553 - recall_at_20: 0.7234 - mrr_at_20: 0.6112 - ndcg_at_20: 0.6369 - map_at_20: 0.6112 - precision_at_20: 0.0362 - regularization_loss: 0.0000e+00 - loss_batch: 2.9622
Epoch 3/5
677/677 [==============================] - 100s 147ms/step - loss: 1.9463 - recall_at_20: 0.8379 - mrr_at_20: 0.7465 - ndcg_at_20: 0.7675 - map_at_20: 0.7465 - precision_at_20: 0.0419 - regularization_loss: 0.0000e+00 - loss_batch: 1.9539
Epoch 4/5
677/677 [==============================] - 100s 148ms/step - loss: 1.5422 - recall_at_20: 0.8713 - mrr_at_20: 0.7875 - ndcg_at_20: 0.8066 - map_at_20: 0.7875 - precision_at_20: 0.0436 - regularization_loss: 0.0000e+00 - loss_batch: 1.5490
Epoch 5/5
84/8

{'loss': 8.851947784423828,
 'recall_at_20': 0.3281925916671753,
 'mrr_at_20': 0.11986491084098816,
 'ndcg_at_20': 0.16598893702030182,
 'map_at_20': 0.11986491084098816,
 'precision_at_20': 0.016409626230597496,
 'regularization_loss': 0.0,
 'loss_batch': 10.054880142211914}

# Run 4

In [9]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_3/sequential_block_19/xl_net_block_3/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 105s 146ms/step - loss: 7.6534 - recall_at_20: 0.1225 - mrr_at_20: 0.0618 - ndcg_at_20: 0.0752 - map_at_20: 0.0618 - precision_at_20: 0.0061 - regularization_loss: 0.0000e+00 - loss_batch: 7.6446
Epoch 2/5
677/677 [==============================] - 100s 147ms/step - loss: 2.9167 - recall_at_20: 0.7304 - mrr_at_20: 0.6236 - ndcg_at_20: 0.6481 - map_at_20: 0.6236 - precision_at_20: 0.0365 - regularization_loss: 0.0000e+00 - loss_batch: 2.9215
Epoch 3/5
677/677 [==============================] - 100s 147ms/step - loss: 1.9719 - recall_at_20: 0.8351 - mrr_at_20: 0.7428 - ndcg_at_20: 0.7639 - map_at_20: 0.7428 - precision_at_20: 0.0418 - regularization_loss: 0.0000e+00 - loss_batch: 1.9820
Epoch 4/5
677/677 [==============================] - 100s 148ms/step - loss: 1.5807 - recall_at_20: 0.8694 - mrr_at_20: 0.7840 - ndcg_at_20: 0.8035 - map_at_20: 0.7840 - precision_at_20: 0.0435 - regularization_loss: 0.0000e+00 - loss_batch: 1.5877
Epoch 5/5
84/8

{'loss': 8.948363304138184,
 'recall_at_20': 0.3253714442253113,
 'mrr_at_20': 0.11741983145475388,
 'ndcg_at_20': 0.16352491080760956,
 'map_at_20': 0.11741983145475388,
 'precision_at_20': 0.016268571838736534,
 'regularization_loss': 0.0,
 'loss_batch': 10.429142951965332}

# Run 5

In [10]:
model_transformer, xlnet_block = get_model()
model_transformer.fit(
    train,
    batch_size=batch_size,
    epochs=n_epoch,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block)
)

predict_last = mm.SequencePredictLast(schema=valid.schema, target=target, transformer=xlnet_block)
model_transformer.evaluate(
    valid,
    batch_size=batch_size,
    pre=predict_last,
    return_dict=True
)

Epoch 1/5


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask_1/GatherV2:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/boolean_mask/GatherV2:0", shape=(None, 192), dtype=float32), dense_shape=Tensor("gradient_tape/model_4/sequential_block_24/xl_net_block_4/prepare_transformer_inputs_4/RaggedToTensor_1/Shape:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


677/677 [==============================] - 105s 147ms/step - loss: 7.2975 - recall_at_20: 0.1426 - mrr_at_20: 0.0798 - ndcg_at_20: 0.0937 - map_at_20: 0.0798 - precision_at_20: 0.0071 - regularization_loss: 0.0000e+00 - loss_batch: 7.2845
Epoch 2/5
677/677 [==============================] - 100s 147ms/step - loss: 2.8422 - recall_at_20: 0.7408 - mrr_at_20: 0.6384 - ndcg_at_20: 0.6618 - map_at_20: 0.6384 - precision_at_20: 0.0370 - regularization_loss: 0.0000e+00 - loss_batch: 2.8481
Epoch 3/5
677/677 [==============================] - 100s 147ms/step - loss: 1.9844 - recall_at_20: 0.8348 - mrr_at_20: 0.7417 - ndcg_at_20: 0.7630 - map_at_20: 0.7417 - precision_at_20: 0.0417 - regularization_loss: 0.0000e+00 - loss_batch: 1.9915
Epoch 4/5
677/677 [==============================] - 100s 147ms/step - loss: 1.5814 - recall_at_20: 0.8700 - mrr_at_20: 0.7848 - ndcg_at_20: 0.8043 - map_at_20: 0.7848 - precision_at_20: 0.0435 - regularization_loss: 0.0000e+00 - loss_batch: 1.5882
Epoch 5/5
84/8

{'loss': 8.888774871826172,
 'recall_at_20': 0.3228324353694916,
 'mrr_at_20': 0.11914832890033722,
 'ndcg_at_20': 0.16426056623458862,
 'map_at_20': 0.11914832890033722,
 'precision_at_20': 0.01614162139594555,
 'regularization_loss': 0.0,
 'loss_batch': 10.727699279785156}